In [20]:
#START → Question Validation → Generate Answer → Summarize Answer → END


!pip install langgraph


In [21]:
!pip install langchain

In [22]:
# we need chatmodel
!pip install langchain-groq

In [23]:
import os
os.environ["GROQ_API_KEY"] = "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"


In [24]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="deepseek-r1-distill-llama-70b" , api_key=os.getenv("GROQ_API_KEY"))

In [25]:
from typing import TypedDict

class SumeryState(TypedDict):
    Question:str
    Answer:str
    Summarize:str

In [26]:
# now we create state
from langgraph.graph import StateGraph

graph = StateGraph(SumeryState)

In [27]:
def Answers(state:SumeryState) -> SumeryState:
  # extract question from llm
  que = state['Question']

  # now create prompt
  prompt = f'write answer on follwing question {que}'

  # genrate answer
  answer = llm.invoke(prompt)

  # update answer in state
  state['Answer'] = answer
  return state

In [28]:
def ansSummarize(state:SumeryState) -> SumeryState:
  # we will extract the answer
  ans = state['Answer']

  #we need to prate summery for that we need to write promopt
  prompt  = f'write summer of the follwing answer {ans}'

  summery = llm.invoke(prompt)

  # lets update summmery on state

  state['Summarize'] = summery

  return state

In [29]:
graph.add_node('Answers', Answers)

graph.add_node("ansSummarize", ansSummarize)



In [30]:
# lets add edge

from langgraph.graph import START , END

graph.add_edge(START,'Answers')
graph.add_edge('Answers', 'ansSummarize')
graph.add_edge('ansSummarize',END)


In [31]:
workflow= graph.compile()

In [33]:
final_state= workflow.invoke({"Question":"who is PM of india"})

In [34]:
final_state

{'Question': 'who is PM of india',
 'Answer': AIMessage(content="<think>\nOkay, so I need to figure out who the Prime Minister of India is. Hmm, I remember hearing about Indian politics a bit, but I'm not exactly sure about the current leader. Let me think. I know that India has a parliamentary system, so the Prime Minister is the head of government. I think the current PM is someone who's been in the news a lot, maybe for a while now.\n\nWait, wasn't there a big election a few years back? I think around 2014 or so. Oh right, that's when Narendra Modi came into power. He was the Chief Minister of Gujarat before that, I believe. So he became the PM after leading the BJP to victory. Since then, he's been re-elected, right? Like in 2019, he won again with a majority. So that means he's still the PM now.\n\nI'm trying to recall if there have been any changes since then. I don't think there's been another election yet, so Modi must still be in office. Yeah, I think that's correct. He's know